In [25]:
import os
os.environ["GEMINI_API_KEY"]="Your Api key"


In [27]:
print(os.environ["GEMINI_API_KEY"])

Your Api Key


In [10]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.
  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastapi-0.115.6-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.32.1-py3-none-any.whl.metadata (6.6 kB)
  Using cached posthog-3.7.4-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached onnxruntime-1.20.1-cp312-cp312-win_amd64.whl.metadata (4.7 kB)
  Using cached opentelemetry_api-1.28.2-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.28.2-py3-non

  You can safely remove it manually.


In [38]:
pip install google-generativeai

  Using cached google_auth-2.36.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached googleapis_common_protos-1.66.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached grpcio-1.68.1-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached protobuf-5.29.1-cp310-abi3-win_amd64.whl.metadata (592 bytes)
   ---------------------------------------- 0.0/760.0 kB ? eta -:--:--
   ---------------------------------------- 760.0/760.0 kB 7.9 MB/s eta 0:00:00
Using cached google_auth-2.36.0-py2.py3-none-any.whl (209 kB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   --- ------------------------------------ 1.0/12.6 MB 10.1 MB/s eta 0:00:02
   ----- ---------------------------------- 1.8/12.6 MB 5.9 MB/s eta 0:00:02
   ------- -------------------------------- 2.4/12.6 MB 4.2 MB/s eta 0:00:03
   -------- ------------------------------- 2.6/12.6 MB 4.2 MB/s eta 0:00:03
   ---------- ---------------------------

In [9]:
from pypdf import PdfReader

def load_pdf(file_path):
    """
    Reads the text content from a PDF file and returns it as a single string.

    Parameters:
    - file_path (str): The file path to the PDF file.

    Returns:
    - str: The concatenated text content of all pages in the PDF.
    """
    # Logic to read pdf
    reader = PdfReader(file_path)

    # Loop over each page and store it in a variable
    text = ""
    for page in reader.pages:
        text += page.extract_text()

    return text

# replace the path with your file path
pdf_text = load_pdf(file_path="C:/Jupiter-notebook/GR5.pdf")

In [17]:
import re
def split_text(text: str):
    """
    Splits a text string into a list of non-empty substrings based on the specified pattern.
    The "\n \n" pattern will split the document para by para
    Parameters:
    - text (str): The input text to be split.

    Returns:
    - List[str]: A list containing non-empty substrings obtained by splitting the input text.

    """
    split_text = re.split('\n \n', text)
    return [i for i in split_text if i != ""]

chunked_text = split_text(text=pdf_text)

In [13]:
import google.generativeai as genai
from chromadb import Documents, EmbeddingFunction, Embeddings
import os

class GeminiEmbeddingFunction(EmbeddingFunction):
    """
    Custom embedding function using the Gemini AI API for document retrieval.

    This class extends the EmbeddingFunction class and implements the __call__ method
    to generate embeddings for a given set of documents using the Gemini AI API.

    Parameters:
    - input (Documents): A collection of documents to be embedded.

    Returns:
    - Embeddings: Embeddings generated for the input documents.
    """
    def __call__(self, input: Documents) -> Embeddings:
        gemini_api_key = os.getenv("GEMINI_API_KEY")
        if not gemini_api_key:
            raise ValueError("Gemini API Key not provided. Please provide GEMINI_API_KEY as an environment variable")
        genai.configure(api_key=gemini_api_key)
        model = "models/embedding-001"
        title = "Custom query"
        return genai.embed_content(model=model,
                                   content=input,
                                   task_type="retrieval_document",
                                   title=title)["embedding"]

In [160]:
import chromadb
from typing import List
def create_chroma_db(documents:List, path:str, name:str):
    """
    Creates a Chroma database using the provided documents, path, and collection name.

    Parameters:
    - documents: An iterable of documents to be added to the Chroma database.
    - path (str): The path where the Chroma database will be stored.
    - name (str): The name of the collection within the Chroma database.

    Returns:
    - Tuple[chromadb.Collection, str]: A tuple containing the created Chroma Collection and its name.
    """
    chroma_client = chromadb.PersistentClient(path=path)
    db = chroma_client.create_collection(name=name, embedding_function=GeminiEmbeddingFunction())

    for i, d in enumerate(documents):
        db.add(documents=d, ids=str(i))

    return db, name

db,name =create_chroma_db(documents="Buddhadeb Karmakar lived in Tasarara. his date of birth is 12/12/1998. ", 
                          path="C:/Repos/RAG/contents", #replace with your path
                          name="buddha")

In [96]:
print(db)

Collection(name=rag_experiment1)


In [162]:
def load_chroma_collection(path, name):
    """
    Loads an existing Chroma collection from the specified path with the given name.

    Parameters:
    - path (str): The path where the Chroma database is stored.
    - name (str): The name of the collection within the Chroma database.

    Returns:
    - chromadb.Collection: The loaded Chroma Collection.
    """
    chroma_client = chromadb.PersistentClient(path=path)
    db = chroma_client.get_collection(name=name, embedding_function=GeminiEmbeddingFunction())

    return db

db=load_chroma_collection(path="C:/Repos/RAG/contents", name="buddha")

In [164]:
def get_relevant_passage(query, db, n_results):
  passage = db.query(query_texts=[query], n_results=n_results)['documents'][0]
  return passage

#Example usage
relevant_text = get_relevant_passage(query="Sanctions on Russia",db=db,n_results=3)
print(relevant_text)

['.', ',', ' ']


In [166]:
def make_rag_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = ("""
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

  ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [168]:
import google.generativeai as genai
def get_answer(prompt):
    gemini_api_key = os.getenv("GEMINI_API_KEY")
    if not gemini_api_key:
        raise ValueError("Gemini API Key not provided. Please provide GEMINI_API_KEY as an environment variable")
    genai.configure(api_key=gemini_api_key)
    model = genai.GenerativeModel('gemini-pro')
    answer = model.generate_content(prompt)
    return answer.text

In [170]:
import google.generativeai as genai
def generate_answer(db,query):
    #retrieve top 3 relevant text chunks
    relevant_text = get_relevant_passage(query,db,n_results=70)
    prompt = make_rag_prompt(query, 
                             relevant_passage="".join(relevant_text)) # joining the relevant chunks to create a single passage
    answer = get_answer(prompt)

    return answer

In [178]:
print(get_relevant_passage("What is the dob of Buuddhadeb Karmakar?: ",db,n_results=50))

['K', 'k', 'b', 'b', 'B', 'd', 'd', 'd', 'd', 'd', 'u', ',', 'T', 'i', 'i', 'i', 'i', 'i', 'm', '9', '9', '1', '1', '1', '8', 't', 't', '2', '2', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'f', 's', 's', 's', 'n', '.', 'l', 'h', 'h', 'h']


In [182]:
# db=load_chroma_collection(path="C:\Repos\RAG\contents", #replace with path of your persistent directory
#                           name="rag_experiment") #replace with the collection name
db=load_chroma_collection(path="C:/Repos/RAG/contents", #replace with path of your persistent directory
                          name="buddha") #replace with the collection name

# answer = generate_answer(db,query="Disadvanteages of  Employee Database And Payroll Management System ")
answer = generate_answer(db,query="What is the dob of Buddhadeb Karmakar?")

print(answer)

Number of requested results 100 is greater than number of elements in index 71, updating n_results = 71


The provided text does not mention anything about Buddhadeb Karmakar's date of birth, so I cannot answer this question from the provided context.
